@meoskis задачка тебе: поработать с датасетом
попробуй загрузи его, проведи его базовый анализ, есть ли там пропуски, наличие спец символов и так далее
ну и почитай как вообще анализировать такие текстовые датасеты, какие там могут быть метрики

## Возникшие вопросы:

1) При анализе комментров следует ли выбрасывать из рассмотрения ссылки (брать только текст комментария). С одной стороны в ссылке может быть нужное решения с правильными словами, когда текст комма будет не полным. С другой как убедится, что она допустим не фишинговая?

## Полезнеые ссылки:

https://towardsdatascience.com/getting-started-with-text-analysis-in-python-ca13590eb4f7

## Информация о датасете:

Данные содержат тренировочную (ranking_train.jsonl) и тестовую (ranking_test.jsonl) выборки.

В каждом файле почти одинаковый формат:
- Отдельная строчка в файле — валидный JSON с семплом
- Каждый JSON имеет поле text с текстом поста и массив сomments из 5 элементов
- Каждый элемент массива это словарь с ключами text с текстом комментария и score с позицией ранжирования (0 cоответствует самому популярному комментарию, 4 самому непопулярному)
- В тестовом файле в поле score стоит null

Заполните тестовую выборку, т.е. расставьте в файле ranking_test.jsonl вместо null значения score. В рамках одного семпла они должны быть от 0 до 4 и не должны повторяться.
Пришлите файл с результатами вместе с кодом и презентацией.

По валидации - будем смотреть на NDCG метрику (hint: она уже есть в библиотеке sklearn). 
В целом, хочется видеть рассуждения, поэтому можно предлагать и другие способы оценки.

## Код

In [1]:
!pip install pyenchant

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 1.5 MB/s eta 0:00:00


In [2]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import copy
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from collections import Counter
import re
import html
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import words
from textblob import TextBlob
from gensim.models import FastText
from sklearn.metrics import ndcg_score
import string
import enchant
from scipy.stats import rankdata

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [12]:
ranking_train = pd.read_json("/content/drive/MyDrive/case/ranking_train.jsonl", lines=True)
ranking_train

,text,comments
0,How many summer Y Combinator fundees decided n...,[{'text': 'Going back to school is not identic...
1,CBS acquires last.fm for $280m,[{'text': 'It will be curious to see where thi...
2,How Costco Became the Anti-Wal-Mart,[{'text': 'I really hate it when people falsel...
3,"Fortune Favors Big Turds | Screw The Money, Th...",[{'text': 'His real point is that something ca...
4,StartupWeekend: 70 Founders Create One Company...,[{'text': 'Looks like someone hasn't read The ...
...,...,...
88102,Don't upgrade to iOS 8.0.1 or you may experien...,[{'text': 'I had this issue and was able to fi...
88103,Ask HN: How do US HNers get their health insur...,[{'text': 'We use a HSA-qualified high-deducti...
88104,Justin Gordon Using React on Rails,[{'text': 'neat insight! A friend of mine conv...
88105,"iOS 8.0.1 released, broken on iPhone 6 models,...","[{'text': 'Ouch, I feel for whoever let this s..."


In [13]:
# Шаг 1. Преобразование датафрейма

ranking_train_exploded = ranking_train.explode('comments')
ranking_train_exploded = pd.concat([ranking_train_exploded.drop(['comments'], axis=1), ranking_train_exploded['comments'].apply(pd.Series).rename(columns={'text':'comments'})], axis=1)

In [14]:
ranking_train_exploded.head(2)

,text,comments,score
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
0,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1


In [17]:
punctuations = string.punctuation
ranking_train_exploded.shape

(440535, 3)

In [ ]:
ranking_train_exploded['polarity'] = ranking_train_exploded['comments'].apply(lambda text: TextBlob(text).sentiment[0])
ranking_train_exploded['subjectivity'] = ranking_train_exploded['comments'].apply(lambda text: TextBlob(text).sentiment[1])
ranking_train_exploded['num_of_symbols'] = ranking_train_exploded['comments'].apply(len)
ranking_train_exploded['num_of_words'] = ranking_train_exploded['comments'].apply(lambda x: len(x.split(sep=' ')))
ranking_train_exploded['num_of_stop_words'] = ranking_train_exploded['comments'].apply(lambda x: len(x.split(sep=' '))-len([word for word in x.split(sep=' ') if word not in stop_words]))
ranking_train_exploded['percent_of_stop_words_with_all_words'] = ranking_train_exploded['num_of_stop_words']/ranking_train_exploded['num_of_words']
ranking_train_exploded['percent_of_len_stop_words'] = 1 - ranking_train_exploded['comments'].apply(lambda x: len(' '.join([word for word in x.split(sep=' ') if word not in stop_words])))/ranking_train_exploded['num_of_symbols']
ranking_train_exploded['avg_len_word'] = ranking_train_exploded['num_of_symbols']/ranking_train_exploded['num_of_words']
ranking_train_exploded = ranking_train_exploded.drop(['num_of_words'], axis=1)
ranking_train_exploded = ranking_train_exploded.drop(['num_of_stop_words'], axis=1)

In [39]:
sentences = test.head(100000)['comments'].to_list()

In [40]:
model_token = FastText(sentences, size=15, window=5, min_count=10, workers=8, sg=1)

In [195]:
model_token.wv['i love VK']

array([ 0.1751606 ,  0.20864713, -0.04136262, -0.34719867,  0.49617186,
       -0.37059578, -0.1569933 , -0.50440276,  0.62647736, -0.14441289,
       -0.10911278, -0.81821746,  0.90729654,  1.0385497 ,  0.04335793],
      dtype=float32)

In [152]:
def vec(i):
  sent = []
  k=0
  for word in df_shuffled['comments'].iloc[i]:
    k+=1
    if word:
      try: 
        sent.append(model_token.wv[word])
      except:
        sent.append(np.zeros(15, dtype=np.float32))
    else:
      sent.append(np.zeros(15, dtype=np.float32))
  return np.mean(sent, axis=0).tolist()

In [ ]:
df_ft = pd.read_csv("/content/drive/MyDrive/case/train_dataset_features.csv").set_index('Unnamed: 0')
df_ft.head(3)

In [22]:
df_ft['punctuation'] = df_ft['comments'].apply(lambda x: sum([1 for char in x if char in punctuations]))/df_ft['num_of_symbols']
df_ft['sentences'] = df_ft['comments'].apply(lambda x: len(nltk.sent_tokenize(x)))/df_ft['num_of_words']
df_ft['connection'] = connect
df_ft['symbols_by_avg'] = df_ft['num_of_symbols']/df_ft['num_of_symbols'].mean()
df_ft['words_by_avg'] = df_ft['num_of_words']/df_ft['num_of_words'].mean()

In [ ]:
connect = []
for i in range(df_ft.shape[0]):
  a = fff(df_ft['comments'].iloc[i])
  b = fff(df_ft['text'].iloc[i])
  connect.append(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
  if i % 30000 == 0:
    print('+30k')

In [43]:
def fff(x):
  sent = []
  for word in x:
    try: 
      sent.append(model_token.wv[word])
    except:
      sent.append(np.zeros(15, dtype=np.float32))
  return np.mean(sent, axis=0)

In [ ]:
df_ft.head(3)

In [54]:
df_ft.to_csv('all_features.csv')

In [16]:
# Шаг 2. Токенезирование строк и удаление стоп-слов и лемитизация слов

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

def word_tokenizer(string):
    string = html.unescape(string.lower())    # Избавляемя от html-формата записи строк
    string = re.sub(r'_', '', string)         # Убираем все нижние подчеркивания
    string = re.sub(r'http\S+', '', string)   # Убираем все ссылки
    string = re.sub(r'\[[0-9]\]', '', string) # Убираем обозначения номеров ссылок?
    
    return [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(string) if not word in stop_words]

test = copy.deepcopy(ranking_train_exploded)
test['text'] = test.apply(lambda row: word_tokenizer(row['text']), axis=1)
test['comments'] = test.apply(lambda row: word_tokenizer(row['comments']), axis=1)

In [178]:
test['polarity'] = df_ft['polarity']
test['subjectivity'] = df_ft['subjectivity']
test['num_of_symbols'] = df_ft['num_of_symbols']
test['percent_of_len_stop_words'] = df_ft['percent_of_len_stop_words']
test['avg_len_word'] = df_ft['avg_len_word']
test['punctuation'] = df_ft['punctuation']
test['sentences'] = df_ft['sentences']
test['connection'] = df_ft['num_of_symbols']
test['symbols_by_avg'] = df_ft['symbols_by_avg']
test['words_by_avg'] = df_ft['words_by_avg']

In [179]:
# разбиваем датафрейм на группы по 5 строк
groups = [test[i:i+5] for i in range(0, len(test), 5)]
new_groups = []
# перемешиваем каждую группу
for group in groups:
    new_groups.append(group.sample(frac=1))

# объединяем группы обратно в один датафрейм
df_shuffled = pd.concat(new_groups)

In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class ResNetBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.activation(out)
        out = self.fc2(out)
        out = out + x
        out = self.activation(out)
        return out

class RankNet(nn.Module):
    def __init__(self, num_inputs, input_dim, hidden_dim, num_outputs):
        super().__init__()
        self.input_dim = input_dim
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.blocks = nn.ModuleList([ResNetBlock(input_dim, hidden_dim) for i in range(num_inputs)])
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim * num_inputs, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)
        
    def forward(self, text_vecs_1, text_vecs_2, text_vecs_3, text_vecs_4, text_vecs_5, features):
        x1 = self.encoder(text_vecs_1)
        x2 = self.encoder(text_vecs_2)
        x3 = self.encoder(text_vecs_3)
        x4 = self.encoder(text_vecs_4)
        x5 = self.encoder(text_vecs_5)
        x = torch.stack((x1, x2, x3, x4, x5), dim=1)
        x = torch.cat((x, features), dim=1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [208]:
k = 88107
txt_try1 = []
for i in range(k):
  post = []
  for j in range(5):
    vec_sent = vec(i*5+j)
    try:
      if vec_sent[0]+1 != vec_sent[0]+1:
        post.append(np.zeros(15, dtype=np.float32).tolist())
      else:
        if type(vec_sent) != list:
          post.append(np.zeros(15, dtype=np.float32).tolist())
        else:
          post.append(vec_sent)
    except:
      post.append(np.zeros(15, dtype=np.float32).tolist())
  txt_try1.append(post)

scores_try1 = [[df_shuffled['score'].iloc[i], df_shuffled['score'].iloc[i+1], df_shuffled['score'].iloc[i+2], df_shuffled['score'].iloc[i+3], df_shuffled['score'].iloc[i+4]] for i in range(0, k*5, 5)]

features_try1 = []
for j in range(0, k*5, 5):
  features_try1.append([[df_shuffled['polarity'].iloc[i], 
                         df_shuffled['subjectivity'].iloc[i], 
                         df_shuffled['num_of_symbols'].iloc[i], 
                         df_shuffled['percent_of_len_stop_words'].iloc[i], 
                         df_shuffled['avg_len_word'].iloc[i], 
                         df_shuffled['punctuation'].iloc[i], 
                         df_shuffled['sentences'].iloc[i], 
                         df_shuffled['connection'].iloc[i], 
                         df_shuffled['symbols_by_avg'].iloc[i], 
                         df_shuffled['words_by_avg'].iloc[i]] for i in range(j, j+5)])

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [128]:
for i in range(len(txt_try1)):
  if len(txt_try1[i]) != 5:
    print('i=', i)
  for j in range(len(txt_try1[i])):
    if len(txt_try1[i][j]) != 15:
      print('i=', i, 'j=', j)

In [209]:
model = RankNet(num_inputs=3, input_dim=15, hidden_dim=5, num_outputs=1)
criterion = nn.MarginRankingLoss(margin=0.1)
optimizer = optim.SGD(model.parameters(), lr=0.001)

texts = txt_try1
scores = scores_try1
features = features_try1

dataset = TensorDataset(torch.tensor(texts), torch.tensor(scores), torch.tensor(features))
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

num_epochs = 3
for epoch in range(num_epochs):
    running_loss = 0.0
    lst_of_ndcg = []
    for batch in dataloader:
        inputs, targets, features = batch
        text_vecs_1, text_vecs_2, text_vecs_3, text_vecs_4, text_vecs_5 = zip(*inputs)
        
        for n in range(len(batch[0])):
          optimizer.zero_grad()
          outputs = model(text_vecs_1[n], 
                          text_vecs_2[n], 
                          text_vecs_3[n], 
                          text_vecs_4[n], 
                          text_vecs_5[n], 
                          features[n].to(torch.float32))
          comments = outputs.view(5)
          comment_pairs = torch.combinations(outputs.view(-1), 2)
          labels = torch.tensor([1 if comments[0] > comments[1] else -1 for i in range(1)] + 
                                [1 if comments[0] > comments[2] else -1 for i in range(1)] + 
                                [1 if comments[0] > comments[3] else -1 for i in range(1)] + 
                                [1 if comments[0] > comments[4] else -1 for i in range(1)] + 
                                [1 if comments[1] > comments[2] else -1 for i in range(1)] + 
                                [1 if comments[1] > comments[3] else -1 for i in range(1)] + 
                                [1 if comments[1] > comments[4] else -1 for i in range(1)] + 
                                [1 if comments[2] > comments[3] else -1 for i in range(1)] + 
                                [1 if comments[2] > comments[4] else -1 for i in range(1)] + 
                                [1 if comments[3] > comments[4] else -1 for i in range(1)])
          loss = criterion(comment_pairs[:, 0], comment_pairs[:, 1], labels.float())
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          lst_of_ndcg.append(ndcg_score([targets[n].tolist()], [comments.tolist()], k=5))
            
    epoch_loss = running_loss / len(dataloader)
    print('Epoch {} loss: {:.4f}'.format(epoch+1, epoch_loss))
    print('NDCG score: ', np.mean(np.array(lst_of_ndcg)), '\n')

Epoch 1 loss: 0.0020
NDCG score:  0.8852600803315848 

Epoch 2 loss: 0.0011
NDCG score:  0.8852755859013778 

Epoch 3 loss: 0.0009
NDCG score:  0.8852619665389597 



In [211]:
torch.save(model.state_dict(), 'model_weights.pth')

In [212]:
queries = scores[:10]

#stroka = 53910

model = RankNet(num_inputs=3, input_dim=15, hidden_dim=5, num_outputs=1)
model.load_state_dict(torch.load("model_weights.pth"))

for stroka in range(10):
  model.eval()
  with torch.no_grad():
      query_scores = model.forward(torch.tensor(txt_try1[stroka][0]),torch.tensor(txt_try1[stroka][1]), torch.tensor(txt_try1[stroka][2]), torch.tensor(txt_try1[stroka][3]), torch.tensor(txt_try1[stroka][4]), torch.tensor(features_try1[stroka], dtype=torch.float32))

  sorted_queries = [query for _, query in sorted(zip(query_scores, queries[stroka]), reverse=True)]

  #print(list(reversed(sorted_queries)))
  print(sorted_queries)

[4, 3, 1, 2, 0]
[3, 4, 2, 1, 0]
[4, 1, 0, 3, 2]
[4, 3, 2, 1, 0]
[0, 4, 3, 2, 1]
[3, 1, 4, 2, 0]
[4, 1, 3, 2, 0]
[1, 0, 2, 4, 3]
[4, 3, 2, 1, 0]
[3, 0, 1, 2, 4]


In [213]:
queries

[[1, 2, 4, 3, 0],
 [3, 2, 4, 1, 0],
 [4, 2, 1, 3, 0],
 [3, 2, 0, 1, 4],
 [4, 1, 2, 0, 3],
 [1, 2, 0, 3, 4],
 [3, 2, 0, 4, 1],
 [4, 0, 2, 3, 1],
 [4, 0, 2, 3, 1],
 [1, 2, 4, 0, 3]]

In [210]:
ranking_test = pd.read_json("/content/drive/MyDrive/case/ranking_test.jsonl", lines=True)

In [226]:
# Шаг 1. Преобразование датафрейма
ranking_test_exploded = ranking_test.explode('comments')
ranking_test_exploded = pd.concat([ranking_test_exploded.drop(['comments'], axis=1), ranking_test_exploded['comments'].apply(pd.Series).rename(columns={'text':'comments'})], axis=1)

ranking_test_exploded['polarity'] = ranking_test_exploded['comments'].apply(lambda text: TextBlob(text).sentiment[0])
ranking_test_exploded['subjectivity'] = ranking_test_exploded['comments'].apply(lambda text: TextBlob(text).sentiment[1])
ranking_test_exploded['num_of_symbols'] = ranking_test_exploded['comments'].apply(len)
ranking_test_exploded['num_of_words'] = ranking_test_exploded['comments'].apply(lambda x: len(x.split(sep=' ')))
ranking_test_exploded['num_of_stop_words'] = ranking_test_exploded['comments'].apply(lambda x: len(x.split(sep=' '))-len([word for word in x.split(sep=' ') if word not in stop_words]))
ranking_test_exploded['percent_of_stop_words_with_all_words'] = ranking_test_exploded['num_of_stop_words']/ranking_test_exploded['num_of_words']
ranking_test_exploded['percent_of_len_stop_words'] = 1 - ranking_test_exploded['comments'].apply(lambda x: len(' '.join([word for word in x.split(sep=' ') if word not in stop_words])))/ranking_test_exploded['num_of_symbols']
ranking_test_exploded['avg_len_word'] = ranking_test_exploded['num_of_symbols']/ranking_test_exploded['num_of_words']

test1 = copy.deepcopy(ranking_test_exploded)

test1['text'] = test1.apply(lambda row: word_tokenizer(row['text']), axis=1)
test1['comments'] = test1.apply(lambda row: word_tokenizer(row['comments']), axis=1)

In [228]:
ranking_test_exploded['punctuation'] = ranking_test_exploded['comments'].apply(lambda x: sum([1 for char in x if char in punctuations]))/ranking_test_exploded['num_of_symbols']
ranking_test_exploded['sentences'] = ranking_test_exploded['comments'].apply(lambda x: len(nltk.sent_tokenize(x)))/ranking_test_exploded['num_of_words']
ranking_test_exploded['symbols_by_avg'] = ranking_test_exploded['num_of_symbols']/ranking_test_exploded['num_of_symbols'].mean()
ranking_test_exploded['words_by_avg'] = ranking_test_exploded['num_of_words']/ranking_test_exploded['num_of_words'].mean()

In [229]:
test1['polarity'] = ranking_test_exploded['polarity']
test1['subjectivity'] = ranking_test_exploded['subjectivity']
test1['num_of_symbols'] = ranking_test_exploded['num_of_symbols']
test1['percent_of_len_stop_words'] = ranking_test_exploded['percent_of_len_stop_words']
test1['avg_len_word'] = ranking_test_exploded['avg_len_word']
test1['punctuation'] = ranking_test_exploded['punctuation']
test1['sentences'] = ranking_test_exploded['sentences']
test1['connection'] = ranking_test_exploded['num_of_symbols']
test1['symbols_by_avg'] = ranking_test_exploded['symbols_by_avg']
test1['words_by_avg'] = ranking_test_exploded['words_by_avg']

In [243]:
test1.to_csv('test_with_features.csv')

In [242]:
def vec_test(i):
  sent = []
  k=0
  for word in test1['comments'].iloc[i]:
    k+=1
    if word:
      try: 
        sent.append(model_token.wv[word])
      except:
        sent.append(np.zeros(15, dtype=np.float32))
    else:
      sent.append(np.zeros(15, dtype=np.float32))
    # if np.array(sent).shape != (k, 15):
    #   print(i)

  return np.mean(sent, axis=0).tolist()

kk = test1.shape[0]
txt_try2 = []
for i in range(int(kk/5)):
  post = []
  for j in range(5):
    vec_sent = vec_test(i*5+j)
    try:
      if vec_sent[0]+1 != vec_sent[0]+1:
        post.append(np.zeros(15, dtype=np.float32).tolist())
      else:
        if type(vec_sent) != list:
          post.append(np.zeros(15, dtype=np.float32).tolist())
        else:
          post.append(vec_sent)
    except:
      post.append(np.zeros(15, dtype=np.float32).tolist())
  txt_try2.append(post)

features_try2 = []
for j in range(0, kk, 5):
  features_try2.append([[test1['polarity'].iloc[i], 
                         test1['subjectivity'].iloc[i], 
                         test1['num_of_symbols'].iloc[i], 
                         test1['percent_of_len_stop_words'].iloc[i], 
                         test1['avg_len_word'].iloc[i], 
                         test1['punctuation'].iloc[i], 
                         test1['sentences'].iloc[i], 
                         test1['connection'].iloc[i], 
                         test1['symbols_by_avg'].iloc[i], 
                         test1['words_by_avg'].iloc[i]] for i in range(j, j+5)])

In [267]:
with open('/content/drive/MyDrive/case/ranking_test.jsonl') as f:
    lines = f.readlines()

In [265]:
model = RankNet(num_inputs=3, input_dim=15, hidden_dim=5, num_outputs=1)
model.load_state_dict(torch.load("model_weights.pth"))

arr = []

for stroka in range(ranking_test.shape[0]):
  model.eval()
  with torch.no_grad():
      query_scores = model.forward(torch.tensor(txt_try2[stroka][0]),torch.tensor(txt_try2[stroka][1]), torch.tensor(txt_try2[stroka][2]), torch.tensor(txt_try2[stroka][3]), torch.tensor(txt_try2[stroka][4]), torch.tensor(features_try2[stroka], dtype=torch.float32))
      nums = query_scores.tolist()
      nums = nums[0]+nums[1]+nums[2]+nums[3]+nums[4]
      x = np.array(nums)
      ranked = rankdata(-x, method='dense') - 1
      scaled = ranked * (4 / np.max(ranked))
      result = scaled.astype(int).tolist()
      arr.append(result)
arr = np.array(arr)

In [268]:
with open('uupdated_file.jsonl', 'w') as f: #создаем новый json, где теперь будут скоры с числами
    for i, line in enumerate(lines):
        obj = json.loads(line.strip())
        for j, score in enumerate(arr[i]):
            obj['comments'][j]['score'] = int(score)
        f.write(json.dumps(obj) + '\n')

In [271]:
# проверка
with open('uupdated_file.jsonl') as f:
    lines = f.readlines()

lines[0]

'{"text": "iOS 8.0.1 released, broken on iPhone 6 models, withdrawn", "comments": [{"text": "I&#x27;m still waiting for them to stabilize wifi on the iPad sith iOS 8. Their quality has really started to slip for me since 10.9 on Mac.", "score": 1}, {"text": "For those who upgraded, no need to do a restore. You can just option-click &quot;Update&quot; and downgrade the OS back to 8.0.0.Instructions: https:&#x2F;&#x2F;gist.github.com&#x2F;locriani&#x2F;f0f5f4f71a28945c3750", "score": 2}, {"text": "Upgraded shortly after it was released and suffered the consequences.  Just was able to restore back down to 8.0I had to turn iMessage off and back on again in order for Apple to re-register my number... until then I was unable to send to any existing contacts.", "score": 3}, {"text": "I think they were under a lot of pressure on the HealthKit front.  That was one of their big flagship iOS 8 features, they got all these app developers to integrate it, then iOS 8 shipped and they had a showstopp